In [14]:
%load_ext autoreload
%autoreload 2
# So code is automatically reloaded when saved in different modules

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [59]:
  
import os
import pickle
import numpy as np
import tensorflow as tf
from scipy import stats
import tensorflow as tf
from sklearn.metrics import roc_auc_score
import time
import logging
import sys
import time
from datetime import datetime
import csv
from collections import Counter, OrderedDict, defaultdict
import sys
sys.path.append('src/taggerSystem/')
from my_util import print_sentence, write_conll, read_conll
from my_data_util import load_and_preprocess_data, load_embeddings, ModelHelper
logger = logging.getLogger("hw3.q2")
logger.setLevel(logging.DEBUG)
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.DEBUG)

## File Paths


In [51]:
data_train = "data/smallIcd9NotesDataTable_test.csv"
data_valid = "data/smallIcd9NotesDataTable_valid.csv"
vocab = "src/taggerSystem/data_hw3_delete/vocab.txt"
wordVecs = "src/taggerSystem/data_hw3_delete/wordVectors.txt"
output_path = 'results/{}/{:%Y%m%d_%H%M%S}/".format(self.cell, datetime.now())'
log_output = output_path + "log"
START_TOKEN = "<s>"
END_TOKEN = "</s>"
NUM = "NNNUMMM"
UNK = "UUUNKKK"
EMBED_SIZE = 50

In [88]:
helper, train, dev, train_raw, dev_raw, xTrain, yTrain, xDev, yDev = load_and_preprocess_data(
    data_train = data_train, data_valid = data_valid)
print('dev raw')
print(dev_raw)
print('dev')
print(dev)
embeddings = load_embeddings(vocab, wordVecs, helper)
helper.save(output_path)# token2id and max length saved to output_path
handler = logging.FileHandler(log_output)
handler.setLevel(logging.DEBUG)
handler.setFormatter(logging.Formatter('%(asctime)s:%(levelname)s: %(message)s'))
logging.getLogger().addHandler(handler)

INFO:Loading training data...
INFO:Done. Read 7 notes
INFO:Loading dev data...
INFO:Done. Read 7 notes
INFO:Total read time 0.008938
INFO:Built dictionary for 15 features.
INFO:There are a total of 4 ICD codes


dev raw
[(['dog', 'dog', 'dog', 'horse'], ['4240', '486']), (['cat', 'cat', 'horse'], ['7212']), (['horse', 'horse', 'horse', 'Batman'], ['4240', '486', '5845']), (['the', 'the', 'of', 'and'], ['4240']), (['the'], ['4240']), (['and'], ['4240']), (['with'], ['4240'])]
dev
[([[2, 9], [2, 9], [2, 9], [1, 9]], array([ 0.,  1.,  0.,  1.])), ([[4, 9], [4, 9], [1, 9]], array([ 0.,  0.,  1.,  0.])), ([[1, 9], [1, 9], [1, 9], [8, 10]], array([ 1.,  1.,  0.,  1.])), ([[3, 9], [3, 9], [6, 9], [5, 9]], array([ 0.,  0.,  0.,  1.])), ([[3, 9]], array([ 0.,  0.,  0.,  1.])), ([[5, 9]], array([ 0.,  0.,  0.,  1.])), ([[7, 9]], array([ 0.,  0.,  0.,  1.]))]


INFO:Initialized embeddings.


## Helper Functions

In [ ]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev = 0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.0, shape = shape)
    return tf.Variable(initial)

def LSTM(x, weight, bias):
    cell = rnn_cell.LSTMCell(n_hidden,state_is_tuple = True)
    multi_layer_cell = tf.nn.rnn_cell.MultiRNNCell([cell] * 2)
    output, state = tf.nn.dynamic_rnn(multi_layer_cell, x, dtype = tf.float32)
    output_flattened = tf.reshape(output, [-1, n_hidden])
    output_logits = tf.add(tf.matmul(output_flattened,weight),bias)
    output_all = tf.nn.sigmoid(output_logits)
    output_reshaped = tf.reshape(output_all,[-1,n_steps,n_classes])
    output_last = tf.gather(tf.transpose(output_reshaped,[1,0,2]), n_steps - 1)  
    #output = tf.transpose(output, [1, 0, 2])
    #last = tf.gather(output, int(output.get_shape()[0]) - 1)
    #output_last = tf.nn.sigmoid(tf.matmul(last, weight) + bias)
    return output_last, output_all

## Hyperparameters

In [34]:
    n_word_features = 1 # Number of features for every word in the input.
    window_size = 1
    n_features = (2 * window_size + 1) * n_word_features # Number of features for every word in the input.
    max_length = 120 # longest sequence to parse
    n_classes = 5
    max_n_labels = 100# max number of labels a note can have. Will be changed later. actually set in data_utils.py
    # above is deprecated. Remove soon
    dropout = 0.5
    embed_size = 50
    hidden_size = 300
    batch_size = 2
    n_epochs = 10
    max_grad_norm = 10.
    lr = 0.001
    tf.reset_default_graph()

    n_input = 1
    n_steps = 10
    n_hidden = 64
    n_classes = 3

    alpha = 0.5

In [ ]:
lstm = tf.contrib.rnn.BasicLSTMCell(lstm_size)
# Initial state of the LSTM memory.
state = tf.zeros([batch_size, lstm.state_size])
probabilities = []
loss = 0.0
for current_batch_of_words in words_in_dataset:
    # The value of state is updated after processing each batch of words.
    output, state = lstm(current_batch_of_words, state)

    # The LSTM output can be used to make next word predictions
    logits = tf.matmul(output, softmax_w) + softmax_b
    probabilities.append(tf.nn.softmax(logits))
    loss += loss_function(probabilities, target_words)

## Extract Data and Word Embeddings

In [79]:
helper, train, dev, train_raw, dev_raw = load_and_preprocess_data(data_train = data_train, 
                                                                  data_valid = data_valid)
embeddings = load_embeddings(vocab, wordVecs, helper)
embed_size = embeddings.shape[1]
helper.save(output_path)# token2id and max length saved to output_path
handler = logging.FileHandler(log_output)
handler.setLevel(logging.DEBUG)
handler.setFormatter(logging.Formatter('%(asctime)s:%(levelname)s: %(message)s'))
logging.getLogger().addHandler(handler)

INFO:Loading training data...
INFO:Done. Read 18 notes
INFO:Loading dev data...
INFO:Done. Read 4 notes
INFO:Total read time 0.017306
INFO:Built dictionary for 7 features.
INFO:There are a total of 4 ICD codes
INFO:Initialized embeddings.


In [ ]:

train_examples = self.preprocess_sequence_data(train_examples_raw)
dev_set = self.preprocess_sequence_data(dev_set_raw)

In [80]:
dev_raw

[(['dog', 'dog', 'dog', 'horse'], ['4240', '486']),
 (['cat', 'cat', 'horse'], ['7212']),
 (['horse', 'horse', 'horse', 'Batman'], ['4240', '486', '5845']),
 (['Batman'], ['4240'])]

In [81]:
dev

[([[2], [2], [2], [1]], array([ 0.,  1.,  1.,  0.])),
 ([[3], [3], [1]], array([ 0.,  0.,  0.,  1.])),
 ([[1], [1], [1], [4]], array([ 1.,  1.,  1.,  0.])),
 ([[4]], array([ 0.,  0.,  1.,  0.]))]